In [28]:
#import modules
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import pickle 

# Charger les données
data = pd.read_csv("card_credit_fraud.csv", delimiter=';')

data = data.dropna()

type_mapping = {'CASHOUT': 1, 'PAYMENT': 2, 'CASHIN': 3, 'TRANSFER': 4, 'DEBIT': 5}
data['type'] = data['type'].map(type_mapping)

# Séparer les caractéristiques (X) de la cible (y)
X = data[['type', 'step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']]
y = data["isFraud"]
seed = 42

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Initialiser le modèle AdaBoost
model = AdaBoostClassifier()

# Créer les stratégies de sous-échantillonnage et de suréchantillonnage
under_sampling = NearMiss(sampling_strategy='auto')
X_under_sample, y_under_sample = under_sampling.fit_resample(X_train, y_train)

over_sampling = SMOTE(sampling_strategy='auto', random_state=seed, k_neighbors=7)
X_train_smote, y_train_smote = over_sampling.fit_resample(X_train, y_train)

# Définir la grille des hyperparamètres à tester
param_grid = {
    'n_estimators': [50, 75, 100],
    'learning_rate': [0.1, 0.5, 1.0]
}

# Initialiser GridSearchCV pour SMOTE avec F1-score comme métrique de scoring
grid_search_smote = GridSearchCV(estimator=AdaBoostClassifier(), param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1)


# Exécuter la recherche sur grille sur les données d'entraînement pour SMOTE
grid_search_smote.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres et le score correspondant pour SMOTE
print("Meilleurs hyperparamètres trouvés pour SMOTE : ", grid_search_smote.best_params_)

# Utiliser le meilleur modèle trouvé pour faire des prédictions sur les données de test pour SMOTE
best_model_smote = grid_search_smote.best_estimator_
y_pred_smote = best_model_smote.predict(X_test)

# Afficher la précision globale pour SMOTE
accuracy_smote = accuracy_score(y_test, y_pred_smote)
print("Précision globale du modèle AdaBoost avec SMOTE : {:.2f}%".format(accuracy_smote * 100))

# Afficher les résultats du suréchantillonnage (SMOTE)
print("\nSuréchantillonnage (SMOTE) :")
print(classification_report(y_test, y_pred_smote, target_names=['Non Fraude', 'Fraude']))

# Matrice de confusion pour le modèle avec suréchantillonnage (SMOTE)
conf_matrix_smote = confusion_matrix(y_test, y_pred_smote)
print("\nMatrice de confusion pour le modèle AdaBoost avec suréchantillonnage (SMOTE) :")
print(conf_matrix_smote)

# Initialiser GridSearchCV pour NearMiss
grid_search_nearmiss = GridSearchCV(estimator=AdaBoostClassifier(), param_grid=param_grid, cv=5, scoring='recall', n_jobs=-1)

# Exécuter la recherche sur grille sur les données d'entraînement pour NearMiss
grid_search_nearmiss.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres et le score correspondant pour NearMiss
print("\nMeilleurs hyperparamètres trouvés pour NearMiss : ", grid_search_nearmiss.best_params_)

# Utiliser le meilleur modèle trouvé pour faire des prédictions sur les données de test pour SMOTE
best_model_nearmiss = grid_search_nearmiss.best_estimator_
y_pred_nearmiss = best_model_nearmiss.predict(X_test)

# Afficher la précision globale pour SMOTE
accuracy_nearmiss = accuracy_score(y_test, y_pred_smote)
print("Précision globale du modèle AdaBoost avec NearMiss : {:.2f}%".format(accuracy_smote * 100))

# Afficher les résultats du suréchantillonnage (SMOTE)
print("\nSouséchantillonnage (NearMiss) :")
print(classification_report(y_test, y_pred_smote, target_names=['Non Fraude', 'Fraude']))

# Matrice de confusion pour le modèle avec suréchantillonnage (SMOTE)
conf_matrix_nearmiss = confusion_matrix(y_test, y_pred_nearmiss)
print("\nMatrice de confusion pour le modèle AdaBoost avec souséchantillonnage (NearMiss) :")
print(conf_matrix_nearmiss)




KeyboardInterrupt: 

In [18]:
from sklearn.metrics import f1_score, roc_auc_score

# Calculer la F-mesure pour SMOTE
f1_smote = f1_score(y_test, y_pred_smote)
print("F-mesure pour le modèle avec SMOTE : {:.2f}".format(f1_smote))

# Calculer l'AUC-ROC pour SMOTE
auc_roc_smote = roc_auc_score(y_test, y_pred_smote)
print("AUC-ROC pour le modèle avec SMOTE : {:.2f}".format(auc_roc_smote))

F-mesure pour le modèle avec SMOTE : 0.96
AUC-ROC pour le modèle avec SMOTE : 0.97


In [16]:
# Enregistrer le meilleur modèle SMOTE avec pickle
with open('best_model_smote.pkl', 'wb') as model_file:
    pickle.dump(best_model_smote, model_file)

# Enregistrer le meilleur modèle NearMiss avec pickle
with open('best_model_nearmiss.pkl', 'wb') as model_file:
    pickle.dump(grid_search_nearmiss.best_estimator_, model_file)


In [ ]:
# Faire des prédictions sur les nouvelles données
predictions = loaded_model.predict(new_data)
